<h2>This notebook studies the <b>Team Orienteering </b> with Pick-up, Delivery Problem and Transfer (TOPDPT)</h2>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

from gurobipy import Model, quicksum, GRB
import numpy as np
from util import generate_node_cargo_size_change, read_pickle, group_cycle_truck, manual_stop
from pathlib import Path

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
num_ins = 10

<h3>How to encode transfer in TOPDPT ?</h3>

Here we consider a simple case of only two trucks $k_1, k_2$.

On top of the single-vehicle Pick-up Delivery Orienteering Problem with Time-Window (PDOTW), we introduce two sets of new variables $z^{kr}_{ij}, u^r_i \in \{0,1\}$.

We add constraints to encode the following situations. 

$$\text{from Jason's code}\left\{\begin{aligned} 
&\sum_{i\in\mathcal{V}}u^r_i,\;  \forall <= 1 &&\text{each cargo } r \text{ can only be transfered at most once}\\
&u^r_i >= \sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ij} - \sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ji},\; \forall i\in\mathcal{V}, r, k && u^r_i = 1 \text{ IFF } \underbrace{\sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ij} = 1}_{\text{ truck }k \text{ arrives node } i \text{ with cargo }r } \text{ and } \underbrace{\sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ji}}_{\text{ truck }k \text{ leaves node } i \text{ without cargo }r }
\end{aligned}\right.$$


\begin{aligned}
&\text{Does not consider transfer}\\
&DT^k_i \ge AT^k_i + \underbrace{\text{fixed time} + \text{load pick-up cargos} + \text{unload delivery cargos}}_{\text{time that } k \text{ spends on node } i\text{, these terms are independent of cargo transfer}}, \; k\in\{k_1,k_2\}, i\in\mathcal{V}\\ \\
&\text{let } T^k = \text{fixed time} + \text{load pick-up cargos} + \text{unload delivery cargos} \\ \\
&\text{Consider transfer from }k_2 \text{ to } k_1\\
&\quad \text{1. if } k_2 \text{ finishes unloading transfer cargos before the arrival of } k_1, \\
&\quad \text{that is, } AT^{k_2}_i + \text{unload }r \le AT^{k_1}_i\\
&\quad DT^{k_1}_i \ge AT^{k_1}_i + \underbrace{\text{load }r}_{\forall u^r_i=1} + T^{k_1}\\
&\quad\quad\quad\quad - M\Big(2 - u^r_i - \sum_{j\in\mathcal{j}:\:j\neq i} z^{k_1r}_{ij}\Big) , \; \forall i\in\mathcal{V}, r
\\ \\
&\quad \text{2. if } k_2 \text{ after } k_1 \text{ and before } k_1 \text{ finished everything} \\
&\quad \text{that is, } AT^{k_1}_i \le AT^{k_2}_i \le AT^{k_1}_i + T^{k_1}\\
&\quad DT^{k_1}_i \ge AT^{k_1}_i + \underbrace{\text{unload and load}r}_{\forall u^r_i=1}   + T\\
&\quad\quad\quad\quad - M\Big(2 - u^r_i - \sum_{j\in\mathcal{j}:\:j\neq i} z^{k_1r}_{ij}\Big) , \; \forall i\in\mathcal{V}, r \\ \\
&\quad \text{3. if } k_2 \text{ arrives after } k_1 \text{ finished everything} \\
&\quad \text{that is, } AT^{k_2}_i \ge AT^{k_1}_i + T^{k_1}\\
&\quad DT^{k_1}_i \ge AT^{k_2}_i + \underbrace{\text{unload and load}r}_{\forall u^r_i=1}   + T\\
&\quad\quad\quad\quad - M\Big(2 - u^r_i - \sum_{j\in\mathcal{j}:\:j\neq i} z^{k_1r}_{ij}\Big) , \; \forall i\in\mathcal{V}, r
\end{aligned}

the last two constraints are controled by Big-M, it is active IFF $\underbrace{u^r_r = 1}_{ r \text{ is transfered at } i}$ and $\underbrace{\sum_{j\in\mathcal{j}:\:j\neq i} z^{kr}_{ij}=1}_{r \text{ is not carried by }k_1 \text{ when arrives at } i}$

In [6]:
def tvopdpt_milp_gurobi(constant, y_sol_,
    selected_cargo, selected_truck, created_truck_yCycle, created_truck_nCycle,
    selected_node, selected_edge, node_cargo_size_change, 
    runtime, filename, verbose = 0):
    
    """
    code a MILP model for two-veihcle orienteering pick-up and delivery problem with transfer (TVOPDPT) in Gurobi
    input:
        constant: dict of parameter values
        y_sol_: solution of y
        selected_cargo: dict of cargos for TVOPDPT
                cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']
        selected_truck: dict of trucks for TVOPDPT, selected_truck = created_truck_yCycle U created_truck_nCycle
                truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']
        created_truck_yCycle: dict of trucks whose origin and destination nodes are the same
        created_truck_nCycle: dict of trucks whose origin and destination nodes are different
        selected_node: list of nodes for TVOPDPT
        selected_edge: dict of edges for TVOPDPT, selected_edge[(node1, node2)]
        node_cargo_size_change: dict of cargo size change at each node
                node_cargo_size_change[(node_, cargo_)] =  0 if node is not cargo origin or destination
                                                        =  cargo_size if n is cargo origin 
                                                        = -cargo_size if n is destination
    return 
        obj_val: objective value
        model_runtime: runtime
        x_sol, z_sol, y_sol, S_sol, D_sol, A_sol, \
       Sb_sol, Db_sol, Ab_sol, u_sol, w_sol
        
    """


    
    if y_sol_ is None:
        model = Model("Gurobi MILP for TVOPDPT")
    else:
        model = Model("Minimize travel cost of TVOPDPT sol")

    def early_termination_callback(model, where):
        if where == GRB.Callback.MIPNODE:
            # Get model objective
            obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)
            if abs(obj - model._cur_obj) > 1e-8:
                # If so, update incumbent and time
                model._cur_obj = obj
                model._time = time.time()
        # Terminate if objective has not improved in 20s
        if time.time() - model._time > 20:
            model._termination_flag = 1
            model.terminate()

    
    ###### Decision Variables ######
    ###### six types of decision variables ######
    
    # binary variables x
    # ==1 if the truck_ traverse an edge 
    x = {}
    for truck_ in selected_truck.keys():
        for i in selected_node:
            for j in selected_node:
                if i != j:
                    x[(i, j, truck_)] = \
                    model.addVar(vtype=GRB.BINARY)
    
    # binary variables y
    # ==1 if cargo_ is carried by truck_
    y={}
    if y_sol_ is None:
        for truck_ in selected_truck.keys():
            for cargo_ in selected_cargo.keys():
                y[(truck_, cargo_)] = model.addVar(vtype=GRB.BINARY)

    else:
        for truck_ in selected_truck.keys():
            for cargo_ in selected_cargo.keys():
                y[(truck_, cargo_)] = y_sol_[(truck_, cargo_)]
    # Integer variables S
    # current total size of cargos on truck_ at node_
    S = {}
    Sb = {}
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            S[(node_, truck_)] = model.addVar(vtype=GRB.INTEGER, lb=0,
                                           ub=selected_truck[truck_][3])
    # if truck_ is a cycle truck and node_ is its destination
    # add a decision variable Sb
    # NOT A D VARIABLE ANYMORE
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Sb[(node_, truck_)] = 0
    
    # integer variable D
    # departure time of truck_ at node_
    D = {}
    Db = {}
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            D[(node_, truck_)] = model.addVar(vtype=GRB.INTEGER, lb=0)
    # if truck_ is a cycle truck and node_ is its destination
    # add a decision variable Ab
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Db[(node_, truck_)] = model.addVar(vtype=GRB.INTEGER, lb=0)
    
    # integer variable A
    # arrival time of truck_ at node_
    A = {}
    Ab = {}
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            A[(node_, truck_)] = model.addVar(vtype=GRB.INTEGER, lb=0)
    # if truck_ is a cycle truck and node_ is its destination
    # add a decision variable Db
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Ab[(node_, truck_)] = model.addVar(vtype=GRB.INTEGER, lb=0)
    

    u, w, z = {}, {}, {}
    for node_ in selected_node:
        for cargo_ in selected_cargo.keys():
            u[(node_, cargo_)] = model.addVar(vtype=GRB.BINARY)
    for node_ in selected_node:
        for cargo_ in selected_cargo.keys():
            w[(node_, cargo_)] = model.addVar(vtype=GRB.BINARY)
    for edge_ in selected_edge.keys():
        for truck_ in selected_truck.keys():
            for cargo_ in selected_cargo.keys():
                if edge_[0] != edge_[1]:
                    z[(edge_[0], edge_[1], truck_, cargo_)] = \
                    model.addVar(vtype=GRB.BINARY)

    ###### Constraints ######
    ###### Distinguish cycle trucks and non-cycle trucks ######
    
    # Flow constraints (3.1)
    # the truck must start from its origin
    for truck_ in selected_truck.keys():
        origin_truck = selected_truck[truck_][0]
        if origin_truck in selected_node:
            model.addConstr( 
                quicksum(x[(origin_truck, succ_node, truck_)] * 1
                         for succ_node in selected_node
                         if succ_node != origin_truck) == 1 
            )
        
    # Flow constraints (3.2)  
    # only applies to non-cycle trucks
    # no flow enters the origin of a non-cycle truck
    for truck_ in created_truck_nCycle.keys():
        origin_truck = created_truck_nCycle[truck_][0]
        if origin_truck in selected_node:
            model.addConstr( 
                quicksum(x[(succ_node, origin_truck, truck_)] * 1
                         for succ_node in selected_node
                         if succ_node != origin_truck) == 0 
            )

    # Flow constraints (3.3)
    # the truck must end at its destination
    for truck_ in selected_truck.keys():
        destination_truck = selected_truck[truck_][1]
        if destination_truck in selected_node:
            model.addConstr( 
                quicksum(x[(pred_node, destination_truck, truck_)] * 1
                         for pred_node in selected_node
                         if pred_node != destination_truck) == 1
            )    
        
    # Flow constraints (3.4)
    # only applies to non-cycle trucks
    # no flow departs from the destination of a non-cycle truck
    for truck_ in created_truck_nCycle.keys():
        destination_truck = created_truck_nCycle[truck_][1]
        if destination_truck in selected_node:
            model.addConstr( 
                quicksum(x[(destination_truck, pred_node, truck_)] * 1
                         for pred_node in selected_node
                         if pred_node != destination_truck) == 0 
            )
    
    
    ### No cycle part below ----------------------------------------------
    
    # Flow constraints (3.5)
    # flow in = flow out
    # Don't consider origin_truck and destination_truck in this constraint
    for truck_ in selected_truck.keys():
        origin_truck = selected_truck[truck_][0]
        destination_truck = selected_truck[truck_][1]
        for node_ in selected_node:
            if node_ != origin_truck and node_ != destination_truck:
                model.addConstr(
                    quicksum(x[(pred_node, node_, truck_)] * 1
                             for pred_node in selected_node
                             if pred_node != node_) 
                    ==
                    quicksum(x[(node_, succ_node, truck_)] * 1
                             for succ_node in selected_node
                             if succ_node != node_) 
                )
    
    # An edge is used at most once by a truck (3.6)
    # only apply for non-cycle trucks
    # and non-origin nodes for cycle trucks
    for truck_ in created_truck_nCycle.keys():
        for i in selected_node:
            for j in selected_node:
                if i != j:
                    model.addConstr(
                        x[(i, j, truck_)] +
                        x[(j, i, truck_)]
                        <= 1
                    )
    for truck_ in created_truck_yCycle.keys():
        origin_truck = created_truck_yCycle[truck_][0]
        for i in selected_node:
            for j in selected_node:
                if i != j:
                    if i != origin_truck and j != origin_truck:
                        model.addConstr(
                            x[(i, j, truck_)] +
                            x[(j, i, truck_)]
                            <= 1
                        )
            
    
    # origin_c is visited by truck_ if y[(truck_, c)] == 1 (3.9)
    for truck_ in selected_truck.keys():
        for cargo_ in selected_cargo.keys():
            origin_cargo = selected_cargo[cargo_][3]
            if origin_cargo in selected_node:
                model.addConstr(
                    quicksum(x[(origin_cargo, node_, truck_)] * 1
                             for node_ in selected_node
                             if node_ != origin_cargo)
                    >= 
                    y[(truck_, cargo_)]
                )
    
    # destination_c is visited by truck_ if y[(truck_, c)] == 1 (3.10)
    for truck_ in selected_truck.keys():
        for cargo_ in selected_cargo.keys():
            destination_cargo = selected_cargo[cargo_][4]
            if destination_cargo in selected_node:
                model.addConstr(
                    quicksum(x[(node_, destination_cargo, truck_)] * 1
                             for node_ in selected_node
                             if node_ != destination_cargo)
                    >= 
                    y[(truck_, cargo_)]
                )
    
    ### Capacity ----------------------------------------------------
    
    # capacity constraints (3.14)
    for truck_ in selected_truck.keys():
        # if truck_ is a NON-cycle truck and node_ is its destination
        # then the truck capacity when departing its destination is 0
        if truck_ in created_truck_nCycle.keys():
            destination_truck = selected_truck[truck_][1]
            model.addConstr(
                S[(destination_truck, truck_)] 
                == 0
            )

    # u-z constraints (New constraints for tvopdpt)
    truck_1, truck_2 = selected_truck.keys()
    # each cargo can be transfered at most once
    for cargo_ in selected_cargo.keys():
        model.addConstr(
            quicksum(u[(node_, cargo_)] * 1 for node_ in selected_node
                     if node_ not in [selected_cargo[cargo_][3], selected_cargo[cargo_][4], selected_truck[truck_1][0], selected_truck[truck_2][1]])
                    <= 1)
        model.addConstr(
            quicksum(w[(node_, cargo_)] * 1 for node_ in selected_node
                     if node_ not in [selected_cargo[cargo_][3], selected_cargo[cargo_][4], selected_truck[truck_2][0], selected_truck[truck_1][1]])
                    <= 1)
        # transfer should not be considered for the following conditions
        # a cargo should not be transfered at cargo's origin and destination nodes,
        # additionally, a truck should not unload cargos at its origin (e.g., transfer cargo from k1 to k2 at k1's origin should be not allowed)
        # a truck should not load cargos at its destination (e.g., transfer cargo from k1 to k2 at k2's destination should be not allowed)
        for node_ in selected_node:
            if node_ not in [selected_cargo[cargo_][3], selected_cargo[cargo_][4], selected_truck[truck_1][0], selected_truck[truck_2][1]]:
                u[(node_, cargo_)] = 0
            if node_ not in [selected_cargo[cargo_][3], selected_cargo[cargo_][4], selected_truck[truck_2][0], selected_truck[truck_1][1]]:
                w[(node_, cargo_)] = 0

    # big m constraint to turn u on and off based on z, u^k_i = 1 IFF 
    for cargo_ in selected_cargo.keys():
        for node_curr in selected_node:    
            if node_curr not in [selected_cargo[cargo_][3], selected_cargo[cargo_][4], selected_truck[truck_1][0], selected_truck[truck_2][1]]:
                # u ^r_i = 1 IFF sum_j z^k1r_{ij} = 1 and sum_l z^k1r_{li} = 0
                # r is transfered from k1 to k2, IFF, r arrives i on k1 and does not leave i on k1
                model.addConstr( u[(node_curr, cargo_)] >= quicksum(z[(node_prev, node_curr, truck_1, cargo_)] * 1
                                                            for node_prev in selected_node if node_prev != node_curr) 
                                                      - quicksum(z[(node_curr, node_next, truck_1, cargo_)] * 1
                                                            for node_next in selected_node if node_next != node_curr)
                            )
            # w ^r_i = 1 IFF sum_j z^k2r_{ij} = 1 and sum_l z^k2r_{li} = 0
            # r is transfered from k2 to k1, IFF, r arrives i on k2 and does not leave i on k2    
            if node_curr not in [selected_cargo[cargo_][3], selected_cargo[cargo_][4], selected_truck[truck_2][0], selected_truck[truck_1][1]]:
                model.addConstr( w[(node_curr, cargo_)] >= quicksum(z[(node_prev, node_curr, truck_2, cargo_)] * 1
                                                            for node_prev in selected_node if node_prev != node_curr)
                                                      - quicksum(z[(node_curr, node_next, truck_2, cargo_)] * 1
                                                            for node_next in selected_node if node_next != node_curr)
                            )

    # Cumulative total size of a truck at a node (3.15)
    # be aware of whether the node is 
    # a cargo origin or cargo destination, or both
    bigM_capacity = 30000
    for truck_ in selected_truck.keys():
        for node1 in selected_node:
            for node2 in selected_node:
                if node1 != node2:
                    # if truck_ is a cycle truck 
                    # and node2 is its destination
                    if truck_ in created_truck_yCycle.keys():
                        if node2 == created_truck_yCycle[truck_][1]:
                            model.addConstr(
                                Sb[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
                        else:
                            model.addConstr(
                                S[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2 \
                                or selected_cargo[cargo_][3] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
                    # else
                    else:
                        if node2 == created_truck_nCycle[truck_][1]:
                            model.addConstr(
                                S[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
                        else:
                            model.addConstr(
                                S[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2 \
                                or selected_cargo[cargo_][3] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
    # Change 20220911 TAN
    # Add total size of cargo <= M * sum_j x^k(i,j)
    for truck_ in selected_truck.keys():
        for node1 in selected_node:
            # if truck_ is a cycle truck 
            # and node2 is its destination
            model.addConstr(
                S[(node1, truck_)]<= 
                bigM_capacity * quicksum(x[(node1, node2, truck_)]\
                        for node2 in selected_node if node1 != node2))

    # total size of cargos at truck origins (3.16)  
    # Should be an equality constraint
    for truck_ in selected_truck.keys():
        origin_truck = selected_truck[truck_][0]
        model.addConstr(
            S[(origin_truck, truck_)] == 
            quicksum(y[(truck_, cargo_)] * \
                     node_cargo_size_change[(origin_truck, cargo_)]
                     for cargo_ in selected_cargo.keys()
                     if selected_cargo[cargo_][3] == origin_truck)
        )
    
    ### Time --------------------------------------------------------
    # The arrival time of a truck at any node (even if not visited) 
    # is less than or equal to the departure time of a truck
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            if truck_ in created_truck_yCycle.keys() and \
               node_ == created_truck_yCycle[truck_][1]:
                model.addConstr(
                    Ab[(node_, truck_)] 
                    <= Db[(node_, truck_)]
                )
                model.addConstr(
                    A[(node_, truck_)] 
                    <= D[(node_, truck_)]
                )
            else:
                model.addConstr(
                    A[(node_, truck_)] 
                    <= D[(node_, truck_)]
                )
    
    # loading and unloading time between arrival and departure (3.17)
    # Don't consider origin_truck in this constraint
    # but for cycle trucks, their origins are also their destinations
    # so we only consider their destination parts
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            # if truck_ is a cycle truck
            if truck_ in created_truck_yCycle.keys():
                # if node_ is its destination
                if node_ == created_truck_yCycle[truck_][1]:
                    model.addConstr( Ab[(node_, truck_)] + constant['node_fixed_time'] 
                                                      + quicksum(y[(truck_, cargo_)] * 
                                                                    int(np.ceil(selected_cargo[cargo_][0] * 
                                                                    constant['loading_variation_coefficient']))
                                                                for cargo_ in selected_cargo.keys()
                                                                if node_ == selected_cargo[cargo_][4])
                                                      + quicksum( (u[node_, cargo_] + w[node_, cargo_] )* 
                                                                    int(np.ceil(selected_cargo[cargo_][0] * 
                                                                    constant['loading_variation_coefficient']))
                                                                for cargo_ in selected_cargo.keys()) 
                                    <= Db[(node_, truck_)]
                                )
                else:
                    model.addConstr( A[(node_, truck_)] + constant['node_fixed_time'] 
                                                     + quicksum(y[(truck_, cargo_)] * 
                                                                    int(np.ceil(selected_cargo[cargo_][0] * 
                                                                    constant['loading_variation_coefficient']))
                                                                for cargo_ in selected_cargo.keys()
                                                                if node_ == selected_cargo[cargo_][3] \
                                                                or node_ == selected_cargo[cargo_][4])
                                                      + quicksum( (u[node_, cargo_] + w[node_, cargo_] )* 
                                                                    int(np.ceil(selected_cargo[cargo_][0] * 
                                                                    constant['loading_variation_coefficient']))
                                                                for cargo_ in selected_cargo.keys()) 
                                    <= D[(node_, truck_)]
                                )
            # if truck_ is a non-cycle truck
            else:
                if node_ != selected_truck[truck_][0]:
                    if node_ == selected_truck[truck_][1]:
                        model.addConstr( A[(node_, truck_)] + constant['node_fixed_time'] 
                                                         + quicksum(y[(truck_, cargo_)] * 
                                                                        int(np.ceil(selected_cargo[cargo_][0] * 
                                                                        constant['loading_variation_coefficient']))
                                                                    for cargo_ in selected_cargo.keys()
                                                                    if node_ == selected_cargo[cargo_][4])
                                                      + quicksum( (u[node_, cargo_] + w[node_, cargo_] )* 
                                                                    int(np.ceil(selected_cargo[cargo_][0] * 
                                                                    constant['loading_variation_coefficient']))
                                                                for cargo_ in selected_cargo.keys()) 
                                        <= D[(node_, truck_)]
                                    )
                    else:
                        model.addConstr( A[(node_, truck_)] + constant['node_fixed_time']
                                                         + quicksum(y[(truck_, cargo_)] * 
                                                                        int(np.ceil(selected_cargo[cargo_][0] * 
                                                                        constant['loading_variation_coefficient']))
                                                                    for cargo_ in selected_cargo.keys()
                                                                    if node_ == selected_cargo[cargo_][3] \
                                                                    or node_ == selected_cargo[cargo_][4])
                                                      + quicksum( (u[node_, cargo_] + w[node_, cargo_] )* 
                                                                    int(np.ceil(selected_cargo[cargo_][0] * 
                                                                    constant['loading_variation_coefficient']))
                                                                for cargo_ in selected_cargo.keys()) 
                                        <= D[(node_, truck_)]
                                    )
    
    # bigM constraints for travel time on edge(i,j) (3.18) 
    # D[prev_node] + edge[(prev_node, curr_node)] <= A[curr_node]
    bigM_time = 2000
    for truck_ in selected_truck.keys():
        for node1 in selected_node:
            for node2 in selected_node:
                if node1 != node2:
                    # if truck_ is a cycle truck and 
                    # node2 is its destination
                    if truck_ in created_truck_yCycle.keys() and \
                       node2 == created_truck_yCycle[truck_][1]:
                        model.addConstr(
                            D[(node1, truck_)] +
                            selected_edge[(node1, node2)]
                            <= 
                            Ab[(node2, truck_)] +
                            bigM_time * (1 - x[(node1, node2, truck_)])
                        )
                    else:
                        model.addConstr(
                            D[(node1, truck_)] +
                            selected_edge[(node1, node2)]
                            <= 
                            A[(node2, truck_)] +
                            bigM_time * (1 - x[(node1, node2, truck_)])
                        )
    
    # Earliest time window of cargos (3.19)
    for truck_ in selected_truck.keys():
        for cargo_ in selected_cargo.keys():
            origin_cargo = selected_cargo[cargo_][3]
            if origin_cargo in selected_node:
                model.addConstr(
                    D[(origin_cargo, truck_)]
                    >= 
                    selected_cargo[cargo_][1] * y[(truck_, cargo_)]
                )
            
    # Latest time window of cargos (3.20)
    for truck_ in selected_truck.keys():
        for cargo_ in selected_cargo.keys():
            destination_cargo = selected_cargo[cargo_][4]
            if destination_cargo in selected_node:
                # if truck_ is a cycle truck and 
                # destination_cargo is its destination
                if truck_ in created_truck_yCycle.keys() and \
                   destination_cargo == created_truck_yCycle[truck_][1]:
                    model.addConstr(
                        Ab[(destination_cargo, truck_)]
                        <= 
                        selected_cargo[cargo_][2] + 
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
                else:
                    model.addConstr(
                        A[(destination_cargo, truck_)]
                        <= 
                        selected_cargo[cargo_][2] + 
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )

    # maximum worktime of trucks (3.21)
    for truck_ in selected_truck.keys():
        origin_truck = selected_truck[truck_][0]
        destination_truck = selected_truck[truck_][1]
        # if truck_ is a cycle truck
        if truck_ in created_truck_yCycle.keys():
            model.addConstr(
                Db[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                <= 
                created_truck_yCycle[truck_][2]  # z[truck_] * 
            )
            model.addConstr(
                Db[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                >= 
                0  # z[truck_] * 
            )
        else:
            model.addConstr(
                D[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                <= 
                selected_truck[truck_][2]  # z[truck_] * 
            )
            model.addConstr(
                D[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                >= 
                0  # z[truck_] * 
            )

    
    # first pickup and then delivery (3.22)
    for truck_ in selected_truck.keys():
        for cargo_ in selected_cargo.keys():
            origin_cargo = selected_cargo[cargo_][3]
            destination_cargo = selected_cargo[cargo_][4]
            if destination_cargo in selected_node:
                # if truck_ is a cycle truck and 
                # destination_cargo is its destination
                if truck_ in created_truck_yCycle.keys() and \
                   destination_cargo == created_truck_yCycle[truck_][1]:
                    model.addConstr(
                        Ab[(destination_cargo, truck_)] - 
                        D[(origin_cargo, truck_)]
                        >= 
                        selected_edge[(origin_cargo, destination_cargo)] -
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
                    model.addConstr(
                        A[(origin_cargo, truck_)] - 
                        D[(destination_cargo, truck_)]
                        >= 
                        selected_edge[(destination_cargo, origin_cargo)] -
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
                else:
                    model.addConstr(
                        A[(destination_cargo, truck_)] - 
                        D[(origin_cargo, truck_)]
                        >= 
                        selected_edge[(origin_cargo, destination_cargo)] -
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
            
            
            
    #


    truck_1, truck_2 = selected_truck.keys()
    for cargo_key in selected_cargo.keys():
        for node_ in selected_node:
            if node_ == selected_truck[truck_1][1] and node_ == selected_truck[truck_2][1]:
                model.addConstr(Ab[(node_, truck_2)]) + 2*quicksum(w[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- w[(node_, cargo_key)]) <= Db[(node_,truck_1)])  

                model.addConstr(Ab[(node_, truck_1)]) + 2*quicksum(u[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- u[(node_, cargo_key)]) <= Db[(node_,truck_2)])  

            if node_ == selected_truck[truck_1][1] and node_ != selected_truck[truck_2][1]:
                model.addConstr(A[(node_, truck_2)]) + 2*quicksum(w[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- w[(node_, cargo_key)]) <= Db[(node_,truck_1)])   

                model.addConstr(Ab[(node_, truck_1)]) + 2*quicksum(u[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- u[(node_, cargo_key)]) <= D[(node_,truck_2)])  

            if node_ != selected_truck[truck_1][1] and node_ == selected_truck[truck_2][1]:
                model.addConstr(Ab[(node_, truck_2)]) + 2*quicksum(w[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- w[(node_, cargo_key)]) <= D[(node_,truck_1)])   

                model.addConstr(A[(node_, truck_1)]) + 2*quicksum(u[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- u[(node_, cargo_key)]) <= Db[(node_,truck_2)])   

            if node_ != selected_truck[truck_1][1] and node_ != selected_truck[truck_2][1]:
                model.addConstr(A[(node_, truck_2)]) + 2*quicksum(w[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- w[(node_, cargo_key)]) <= D[(node_,truck_1)])  

                model.addConstr(A[(node_, truck_1)]) + 2*quicksum(u[node_, cargo_] for cargo_ in selected_cargo.keys() 
                             - bigM_time* (1- u[(node_, cargo_key)]) <= D[(node_,truck_2)])   

                            
          
    

    if y_sol_ is None:
        
        # cargo number cost: proportional to the number of cargo carried by the only truck
        cost_cargo_number = quicksum(y[truck_, cargo_] * 
                                    constant['truck_running_cost'] * 1000
                                    for truck_ in selected_truck.keys()
                                    for cargo_ in selected_cargo.keys())
        
        model.setObjective(cost_cargo_number)

        model.modelSense = GRB.MAXIMIZE
        # set Params.Heuristics to 0.5 
        # such that it better finds feasible solution
        model.Params.Heuristics = 0.5
        model.Params.LogFile = filename
        callback=early_termination_callback

    else:
        cost_travel = quicksum(x[(node1, node2, truck_)] * 
                            selected_edge[(node1, node2)] * 
                            constant['truck_running_cost']
                            for truck_ in selected_truck.keys()
                            for node1 in selected_node
                            for node2 in selected_node
                            if node1 != node2)
        model.setObjective(cost_travel)
        model.modelSense = GRB.MINIMIZE
        model.Params.LogFile = filename[:-4]+'_reopt.log'
        callback=None
    
    ###### Integrate the model and optimize ######

    # private parameters to help with callback function
    model.Params.LogToConsole  = 0
    model._cur_obj = float('inf')
    model._time = time.time()
    model._no_improve_iter = 0
    model._termination_flag = 0

    model.Params.timeLimit = runtime
    model.Params.OutputFlag = 1
    model.update()

    if callback is None:
        model.optimize()
    else:
        if verbose >0:
            print('Use soft-termination through callback, terminate if no better solution in 20 s')
        model.optimize(callback=callback)

    # if infeasible
    if model.Status == 3:
        if verbose >0: print('+++ MIP [Infeasible Proved] ')
        return -1, runtime, *[[]]*11
    
    model_runtime = model.Runtime
    obj_val = model.objVal

    
    # if no objective value
    if float('inf') == obj_val:
        if verbose >0: print('+++ MIP [Infeasible] ')
        return -1, runtime, *[[]]*11
        
    
    if verbose > 0:
        print(f'+++ {model.ModelName} [Feasible] ')
        if model._termination_flag == 1:
            print('    soft termination: failed to improve best solution for 20s.')
        elif model._termination_flag == 2:
            print('    soft termination: failed to improve obj for 50 consecutive feasible solutions.') 
        if verbose > 1:
            print("   [Gurobi obj value] is %i" % obj_val)
            print("   [Gurobi runtime] is %f" % model_runtime)
    
    
    ###### Get solutions ######
    
    # store all values in a list: sol
    sol = []
    for ss in model.getVars():
        sol.append(int(ss.x))
        
    # retrieve values from the list sol
    count = 0
    # binary variables x
    x_sol = {}
    for truck_ in selected_truck.keys():
        for i in selected_node:
            for j in selected_node:
                if i != j:
                    x_sol[(i, j, truck_)] = sol[count]
                    count += 1
                
    # binary variables y
    if y_sol_ is None:
        y_sol = {}
        for truck_ in selected_truck.keys():
            for cargo_ in selected_cargo.keys():
                y_sol[(truck_, cargo_)] = sol[count]
                count += 1
    else:
        y_sol = y_sol_.copy() # for the convinience of the printout below
    
    # integer variable S
    S_sol = {}
    Sb_sol = {}
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            S_sol[(node_, truck_)] = sol[count]
            count += 1
    # if truck_ is a cycle truck and node_ is its destination
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Sb_sol[(node_, truck_)] = 0
            
    # integer variable D
    D_sol = {}
    Db_sol = {}
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            D_sol[(node_, truck_)] = sol[count]
            count += 1
    # if truck_ is a cycle truck and node_ is its destination
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Db_sol[(node_, truck_)] = sol[count]
        count += 1
    
    # integer variable A
    A_sol = {}
    Ab_sol = {}
    for truck_ in selected_truck.keys():
        for node_ in selected_node:
            A_sol[(node_, truck_)] = sol[count]
            count += 1
    # if truck_ is a cycle truck and node_ is its destination
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Ab_sol[(node_, truck_)] = sol[count]
        count += 1
    
    u_sol, w_sol, z_sol = {}, {}, {}
    for node_ in selected_node:
        for cargo_ in selected_cargo.keys():
            u_sol[(node_, cargo_)] = sol[count]
    for node_ in selected_node:
        for cargo_ in selected_cargo.keys():
            w_sol[(node_, cargo_)] = sol[count]
    for edge_ in selected_edge.keys():
        for truck_ in selected_truck.keys():
            for cargo_ in selected_cargo.keys():
                if edge_[0] != edge_[1]:
                    z_sol[(edge_[0], edge_[1], truck_, cargo_)] = sol[count]

    milp_sol = (x_sol, z_sol, y_sol, S_sol, D_sol, A_sol, Sb_sol, Db_sol, Ab_sol, u_sol, w_sol)

    return obj_val, model_runtime, milp_sol

In [7]:
def solve_pdotw_mip(ins,  # dict contains the data of pdpt instance,
                    path_, # file where all data of pdotw solutions are saved
                    optimize_pdotw_routes = True,
                    max_runtime = 100,
                    verbose = 0):  

    # load data from ins
    selected_truck = ins['truck']
    selected_cargo = ins['cargo']
    selected_node = ins['nodes']
    selected_edge = ins['edge_shortest']    
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']


    # edge_shortest, path_shortest = replace_edge_by_shortest_length_nx(nodes, edges)
    # single_truck_deviation = calculate_single_truck_deviation(truck, cargo, edge_shortest)
    
    start_time = time.time()
    print(f'========= START [PDOTW with truck ========= ')

    
    created_truck = selected_truck.copy()
    
    # nodes in the cluster
    # Note. cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time','departure_node', 'arrival_node']
    # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']
    

    if verbose >0:
        print(f'+++ Preprocess data to instantiate a PDOTW MIP')
    if verbose > 2:
        print(f'    [selected_cargo] size: {len(selected_cargo)}')
        for key, value in selected_cargo.items():
            print(f'        {key, value}')
        print(f'    [created_truck] size: {len(created_truck)}')
        for key, value in created_truck.items():
            print(f'       {key, value}')
    
    ### Need to update node_cargo_size_change 
    node_cargo_size_change = generate_node_cargo_size_change(selected_node, selected_cargo)

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, selected_truck = group_cycle_truck(created_truck)  
    
    if verbose > 2:
        print('    [created_truck_yCycle]', created_truck_yCycle)
        print('    [created_truck_nCycle]', created_truck_nCycle)


    if verbose >0:
        print(f'+++ Solve TVOPDPT MILP in Gurobi')
    ### use gurobi to solve the GROW origin PDPTW
    # Note. the pdotw_mip_gurobi function is desgined to take the same arguments as pdpt function
    # but some parameters
    gurobi_log_file = os.path.join(path_, f'_gurobi.log')


    obj_val, model_runtime, milp_sol = tvopdpt_milp_gurobi(constant, None, 
                                                           selected_cargo, 
                                                           selected_truck, 
                                                           created_truck_yCycle, 
                                                           created_truck_nCycle,
                                                           selected_node, 
                                                           selected_edge, 
                                                           node_cargo_size_change, 
                                                           max_runtime, 
                                                           gurobi_log_file, 
                                                           verbose = 0)

    x_sol, z_sol, y_sol, S_sol, D_sol, A_sol, Sb_sol, Db_sol, Ab_sol, u_sol, w_sol = milp_sol
    # Index k for truck is pre-defined
    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #z_sol: z^{kr}_{ij}, if truck k visit edge (i,j) with cargo r
    #u_sol: u^r_i, if cargo r is transfered at node i
    #y_sol: y^k_r, if parcel r is carried by truck k
    #S_sol: x^k_i, total size of cargos on truck k at node i
    #D_sol: D^k_i, depature time of truck k at node i
    #A_sol: A^k_i, arrival time of truck k at node i

    res = {'x_sol':  x_sol,
           'z_sol':  z_sol,
           'y_sol':  y_sol,
           'S_sol':  S_sol,
           'D_sol':  D_sol,
           'A_sol':  A_sol,
           'Sb_sol': Sb_sol,
           'Db_sol': Db_sol,
           'Ab_sol': Ab_sol,
           'u_sol':  u_sol,
           'w_sol':  w_sol
          }

    return res

In [9]:

def main():
    case_num, ins_idx, num_trucks = 1, 0, 2
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
    print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
    pdpt_ins = read_pickle(pdpt_ins_filename)
    

    path_ = os.path.join(dir_, 'toy/top_pdpt')
    Path(path_).mkdir(parents=True, exist_ok=True)
    res = solve_pdotw_mip(pdpt_ins,  # dict contains the data of pdpt instance,
                          path_, # file where all data of pdotw solutions are saved
                          optimize_pdotw_routes = True,
                          max_runtime = 100,
                          verbose = 0)

    manual_stop()

main()

===== START team orienteering pdpt
      /home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins1.pkl


FileNotFoundError: [Errno 2] No such file or directory: '/home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins1.pkl'

In [ ]:

# def main_toy():
#     pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
#     print(f'===== START route+schedule\n      {pdpt_ins_filename}')
#     pdpt_ins = read_pickle(pdpt_ins_filename)

#     path_ = os.path.join('/home/tan/Documents/GitHub/pdpt_2022/', 'out/top_pdpt')
#     Path(path_).mkdir(parents=True, exist_ok=True)
#     res = solve_pdotw_mip(ins = pdpt_ins,  # dict contains the data of pdpt instance,
#                           path_ = path_, # file where all data of pdotw solutions are saved
#                           optimize_pdotw_routes = True,
#                           verbose = 0)

#     manual_stop()

# main(case_num=1, num_trucks = 2, ins_idx = 1)

In [ ]:
print(*[[]]*11)